In [1]:
import or_suite
import numpy as np
import gym

CONFIG = or_suite.envs.env_configs.rideshare_graph_0_3_rides_config

In [2]:
# making an instance of the environment
env = gym.make('Rideshare-v0', config=CONFIG)

In [3]:
print(env.state)
print(env.action_space)
print(env.observation_space)

[1000    0    0    0    3    3]
Discrete(4)
MultiDiscrete([1000 1000 1000 1000    4    4])


In [4]:
print(env.gamma)

0.25


The cell below displays how the probability of accepting a ride changes based on the distance from the car to the request.

We first dispatch 1000 cars to satisfy a request that is 1 unit away, then 5 units away, and finally 10 units away.

In [5]:
# 1000 rides with dispatch length of 1 unit
env = gym.make('Rideshare-v0', config=or_suite.envs.env_configs.rideshare_graph_0_1_rides_config)
for i in range(1000):
    newState, reward, done, info = env.step(0)
print("dispatch lengths of 1 unit:")
print(env.state)
print("\n")

# 1000 rides with dispatch length of 5 units
env = gym.make('Rideshare-v0', config=or_suite.envs.env_configs.rideshare_graph_0_2_rides_config)
for i in range(1000):
    newState, reward, done, info = env.step(0)
print("dispatch lengths of 5 units:")
print(env.state)
print("\n")

# 1000 rides with dispatch length of 10 units
env = gym.make('Rideshare-v0', config=or_suite.envs.env_configs.rideshare_graph_0_3_rides_config)
for i in range(1000):
    newState, reward, done, info = env.step(0)
print("dispatch lengths of 10 units:")
print(env.state)
print("\n")


dispatch lengths of 1 unit:
[357 643   0   0   1   1]


dispatch lengths of 5 units:
[624   0 376   0   2   2]


dispatch lengths of 10 units:
[847   0   0 153   3   3]




In [7]:
done_v = False
env = gym.make('Rideshare-v0', config=CONFIG)

for i in range(1000):
    if done_v == False:
        newState, reward, done, info = env.step(0)
        if done == True:
            done_v = True
        if i%50 == 0:
            print(env.state)
        
print(env.state)

[999   0   0   1   3   3]
[994   0   0   6   3   3]
[983   0   0  17   3   3]
[980   0   0  20   3   3]
[971   0   0  29   3   3]
[963   0   0  37   3   3]
[956   0   0  44   3   3]
[949   0   0  51   3   3]
[941   0   0  59   3   3]
[935   0   0  65   3   3]
[929   0   0  71   3   3]
[920   0   0  80   3   3]
[913   0   0  87   3   3]
[906   0   0  94   3   3]
[898   0   0 102   3   3]
[892   0   0 108   3   3]
[882   0   0 118   3   3]
[875   0   0 125   3   3]
[873   0   0 127   3   3]
[865   0   0 135   3   3]
[857   0   0 143   3   3]


In [10]:
# testing the step function
print(env.state)
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

[99  0  0  1  2  2]
prob: 0.5
accept: 1
accept service
DONE DONE DONE
[98  0  1  1  2  0]
-2.23606797749979
True


In [6]:
newState, reward,  done, info = env.step(1)
print(newState)
print(reward)
print(done)

0.5 0
decline service
[0 2 3 5 3 3]
-10000
False


In [8]:
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

[0 2 3 5 2 2]
-10000
True


In [9]:
newState, reward,  done, info = env.step(0)
print(newState)
print(reward)
print(done)

[0 2 3 5 1 0]
-10000
True


### Step 1: Import Required Packages

The main package for ORSuite is contained in `or_suite`.  However, some additional packages may be required for specific environments / algorithms.  Here, we include `stable baselines`, a package containing implementation for state of the art deep RL algorithms, and `matploblib` for the plotting.

In [8]:
import or_suite
import gym
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
import numpy as np

### Step 2: Pick problem parameters for the environment

Here we use the ambulance metric environment as outlined in `or_suite/envs/ambulance/ambulance_metric.py`.  The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use one the default for the ambulance problem in a metric space.

In addition, we need to specify the number of episodes for learning, and the number of iterations (in order to plot average results with confidence intervals).

In [9]:
DEFAULT_CONFIG = or_suite.envs.env_configs.rideshare_graph_default_config
epLen = DEFAULT_CONFIG['epLen']
nEps = 200
numIters = 5

### Step 3: Pick simulation parameters

Next we need to specify parameters for the simulation.  This includes setting a seed, the frequency to record the metrics, directory path for saving the data files, a deBug mode which prints the trajectory, etc.

In [10]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': './data/rideshare/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters,
                    'render': False,
                    'saveTrajectory': True, 
                    'epLen' : 5}

rideshare_env = gym.make('Rideshare-v0', config=DEFAULT_CONFIG)
mon_env = Monitor(rideshare_env)

### Step 4: Pick list of algorithms

We have several heuristics implemented for each of the environments defined, in addition to a `random` policy, and some `RL discretization based` algorithms.  Here we pick a couple of the heuristics, and a PPO algorithm implemented from `stable baselines` just to test.

In [11]:
agents = {'Random': or_suite.agents.rl.random.randomAgent(),
          'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
          }

/Users/maxsolberg/opt/anaconda3/envs/ORSuite/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:131: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a multiple of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  warnings.warn(


### Step 5: Run simulations

In [12]:
#or_suite.utils.run_single_algo(mon_env, agents['Random'], DEFAULT_SETTINGS)

In [13]:
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = './data/vaccine_metric_test_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(mon_env, agents[agent], DEFAULT_SETTINGS)

Random
**************************************************
Running experiment
**************************************************
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 0
decline service
0.5 1
accept service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline s

3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 1
accept service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 

3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
declin

0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 1
accept service
0.5 0
dec

3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
0.5 1
accept service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decli

...

0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 1
accept service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 1
accept service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decli

3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 1
accept service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
de

3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept 

3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.720075976020836e-44 0
decline service
0.5 0
decline service
0.5 0
decline service
0.5 1
accept service
3.720075976020836e-44 0
decline service
0.5 0
decline service
3.

ValueError: arrays must all be same length

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(rideshare_env, skip_render_check=True)